### OpenFIBSEM DB

sqlite3 database for storing metadata about FIBSEM datasets.

https://www.sqlitetutorial.net/sqlite-python

In [ ]:
%load_ext autoreload
%autoreload 2


import sqlite3
from fibsem.db.util import *

### Create TABLE


In [ ]:
# create a database connection
conn = create_connection(database)

# create tables
if conn is not None:
    # create projects table
    create_table(conn, sql_create_projects_table)

    # create tasks table
    create_table(conn, sql_create_tasks_table)
else:
    print("Error! cannot create the database connection.")


#### INSERT DATA

In [ ]:
import sqlite3
from sqlite3 import Error




# create a database connection
conn = create_connection(database)
with conn:
    # create a new project
    project = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
    project_id = create_project(conn, project)

    # tasks
    task_1 = ('Analyze the requirements of the app', 1, 1, project_id, '2015-01-01', '2015-01-02')
    task_2 = ('Confirm with user about the top requirements', 1, 1, project_id, '2015-01-03', '2015-01-05')

    # create tasks
    create_task(conn, task_1)
    create_task(conn, task_2)



### Update Task

In [ ]:

# create a database connection
conn = create_connection(database)
with conn:
    update_task(conn, (2, '2015-01-04', '2015-01-06', 2))


### Delete Task


In [ ]:
conn = create_connection(database)
with conn:
    delete_task(conn, 1);
    # delete_all_tasks(conn);

### Query Data


In [ ]:
# create a database connection
conn = create_connection(database)
with conn:
    print("1. Query task by priority:")
    select_task_by_priority(conn, 1)

    print("2. Query all tasks")
    select_all_tasks(conn)

## PANDAS


In [ ]:
import pandas as pd

conn = create_connection(database)
df = pd. read_sql('SELECT * from tasks', conn)

In [ ]:
display(df)

In [ ]:
# write the data to a sqlite table

df = pd.DataFrame({"id": 5, "name": "test", "priority": 3, "status_id": 3, "project_id": 3, "begin_date": '2016-01-12', "end_date": '2016-02-24'}, 
index=[0])

display(df)
df.to_sql('tasks', conn, if_exists='append', index = False)

In [ ]:
df = pd. read_sql('SELECT * from tasks', conn)

In [ ]:
display(df)

DATABASE

--- EXPERIMENT MANAGMENT ---

project
- id
- name
- user_id

user:
- id
- name

experiment
- id
- name
- project_id
- user_id
- sample_id
- path

sample
- id
- name



---- Analytics ----

ml
detection
alignment
interaction
history
steps


In [ ]:

# MILESTONE 1
# TODO: initialise database script
# TODO: migrate ml data collection to store in db
# TODO: setup projects, experiment, user tables
# TODO: separate ETL from statistics / analytics application
# TODO: enable user to select project, experiment
# TODO: enable multi-experiment analytics
# TODO: fix experiment to dataframe function, make actually useful


In [ ]:
%load_ext autoreload
%autoreload 2

import datetime
import sqlite3
from fibsem.db.util import *


In [ ]:
SQL_CREATE_PROJECTS_TABLE = """ CREATE TABLE IF NOT EXISTS projects (
                                    id INTEGER PRIMARY KEY,
                                    name VARCHAR(100) NOT NULL,
                                    date TIMESTAMP NOT NULL,
                                    user VARCHAR(100) NOT NULL
                                ); """

SQL_CREATE_USERS_TABLES = """CREATE TABLE IF NOT EXISTS users (
                                id INTEGER PRIMARY KEY,
                                name VARCHAR(100) NOT NULL,
                                email VARCHAR(100) NOT NULL,
                                password VARCHAR(100) NOT NULL
                            );"""


SQL_CREATE_SAMPLES_TABLE = """CREATE TABLE IF NOT EXISTS samples (
                                id INTEGER PRIMARY KEY,
                                name VARCHAR(100) NOT NULL,
                                project_id INTEGER NOT NULL,
                                date TIMESTAMP NOT NULL,
                                user VARCHAR(100) NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                                );"""


SQL_CREATE_EXPERIMENTS_TABLE = """CREATE TABLE IF NOT EXISTS experiments (
                                id INTEGER PRIMARY KEY,
                                name VARCHAR(100) NOT NULL,
                                project_id INTEGER NOT NULL,
                                date TIMESTAMP NOT NULL,
                                user VARCHAR(100) NOT NULL,
                                sample_id INTEGER NOT NULL,
                                program VARCHAR(100) NOT NULL,
                                method VARCHAR(100) NOT NULL,
                                path VARCHAR(100) NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                                FOREIGN KEY (sample_id) REFERENCES samples (id)
                            );"""


# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_PROJECTS_TABLE)
create_table(conn, SQL_CREATE_USERS_TABLES)
create_table(conn, SQL_CREATE_SAMPLES_TABLE)
create_table(conn, SQL_CREATE_EXPERIMENTS_TABLE)




### Create Project

In [ ]:
import datetime

# create_project(conn, 
#     ("WAFFLE-METHOD-DEVELOPMENT", datetime.datetime.now(), "patrick")
# )

### CREATE USER

In [ ]:
create_user(conn, 
    ('hannah', 'hannah.siems@monash.edu', 'password')
)

### CREATE SAMPLE

In [ ]:
create_sample(
    conn, 
    ("WAFFLE-01", 1, datetime.datetime.now(), "hannah")
)

### CREATE EXPERIMENT



In [ ]:
create_experiment(conn,

    ("WAFFLE-15082023", 1, datetime.datetime.now(), "hannah", 1)

)

In [ ]:
import pandas as pd
print("PROJECTS")
df = pd. read_sql('SELECT * from projects', conn)
display(df)

print("USERS")
df = pd. read_sql('SELECT * from users', conn)
display(df)

print("SAMPLES")
df = pd. read_sql('SELECT * from samples', conn)
display(df)

print("EXPERIMENTS")
df = pd. read_sql('SELECT * from experiments', conn)
display(df)

In [ ]:
### REPLACE ? --> UPSERT?

# might not want to use, replaces experiment?

### Sync Experiments


In [ ]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]

for PATH in EXPERIMENTS:
    
    print(PATH)   

    exp = Experiment.load(os.path.join(PATH, "experiment.yaml"))
    df = exp.to_dataframe_v2()

    # format date as datetime
    df["date"] = pd.to_datetime(df["date"], unit="s")
    df["date"] = pd.to_datetime(df["date"], utc=True)

    # drop experiment_id
    df = df.drop(columns=["experiment_id", "num_lamella"])
    df["project_id"] = 1
    df["sample_id"] = 1
    df["user"] = "hannah"
    df["method"] = "waffle"
    
    display(df)

    df.to_sql('experiments', conn, if_exists='append', index = False)

    print("------------------")





In [ ]:
df = pd.read_sql("SELECT * FROM [experiments]", conn)
display(df)

In [ ]:

PROJECT_NAME = "WAFFLE-METHOD-DEVELOPMENT"

df = pd.read_sql(f"SELECT * FROM projects WHERE name='{PROJECT_NAME}'", conn)
PROJECT_IDS = df["id"].values

df = pd.read_sql(f"SELECT * FROM experiments WHERE project_id={PROJECT_IDS[0]}", conn)

display(df)

In [ ]:

SQL_CREATE_HISTORY_TABLE = """CREATE TABLE IF NOT EXISTS history (
                                id INTEGER PRIMARY KEY,
                                petname VARCHAR(100) NOT NULL,
                                stage VARCHAR(100) NOT NULL,
                                start TIMESTAMP NOT NULL,
                                end TIMESTAMP NOT NULL,
                                duration FLOAT NOT NULL,
                                experiment_id INTEGER NOT NULL,
                                FOREIGN KEY (experiment_id) REFERENCES experiment (id)
                                
                            );"""


# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_PROJECTS_TABLE)

In [ ]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]


df_exp = pd.read_sql("SELECT * FROM experiments", conn)
display(df_exp)

for PATH in EXPERIMENTS:
    
    print(PATH)   

    dfs = calculate_statistics_dataframe(PATH)
    df = dfs[1]

    # format date as datetime
    df["start"] = pd.to_datetime(df["start"], unit="s")
    df["start"] = pd.to_datetime(df["start"], utc=True)

    df["end"] = pd.to_datetime(df["start"], unit="s")
    df["end"] = pd.to_datetime(df["start"], utc=True)


    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])
    
    display(df)


    df.to_sql('history', conn, if_exists='append', index = False)

    print("------------------")

In [ ]:


SQL_CREATE_STEPS_TABLE = """CREATE TABLE IF NOT EXISTS steps (
                                id INTEGER PRIMARY KEY,
                                petname VARCHAR(100) NOT NULL,
                                stage VARCHAR(100) NOT NULL,
                                step VARCHAR(100) NOT NULL,
                                step_n INTEGER NOT NULL,
                                timestamp TIMESTAMP NOT NULL,
                                duration FLOAT NOT NULL,
                                experiment_id INTEGER NOT NULL,
                                FOREIGN KEY (experiment_id) REFERENCES experiment (id)
                                
                            );"""

# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_STEPS_TABLE)

In [ ]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]


df_exp = pd.read_sql("SELECT * FROM experiments", conn)
display(df_exp)

for PATH in EXPERIMENTS:

    dfs = calculate_statistics_dataframe(PATH)
    df = dfs[3]

    # display(df)
    # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())

    df.to_sql('steps', conn, if_exists='append', index = False)

    print("------------------")

In [ ]:
SQL_CREATE_DETECTIONS_TABLE = """CREATE TABLE IF NOT EXISTS detections (
                                id INTEGER PRIMARY KEY,
                                petname VARCHAR(100) NOT NULL,
                                stage VARCHAR(100) NOT NULL,
                                step VARCHAR(100) NOT NULL,
                                feature VARCHAR(100) NOT NULL,
                                dpx_x INTEGER NOT NULL,
                                dpx_y INTEGER NOT NULL,
                                dm_x FLOAT NOT NULL,
                                dm_y FLOAT NOT NULL,
                                is_correct BOOL NOT NULL,
                                beam_type VARCHAR(100) NOT NULL,
                                fname VARCHAR(100) NOT NULL,
                                timestamp TIMESTAMP NOT NULL,
                                experiment_id INTEGER NOT NULL,
                                FOREIGN KEY (experiment_id) REFERENCES experiment (id)
                                
                            );"""

# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_DETECTIONS_TABLE)

In [ ]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]


df_exp = pd.read_sql("SELECT * FROM experiments", conn)
display(df_exp)


# 4, stage
# 5, detections
# 6, interactions

for PATH in EXPERIMENTS:

    dfs = calculate_statistics_dataframe(PATH)
    df = dfs[5]




    # # display(df)
    # # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())

    df.to_sql('detections', conn, if_exists='append', index = False)

    print("------------------")

In [ ]:
SQL_CREATE_INTERACTIONS_TABLE = """CREATE TABLE IF NOT EXISTS interactions (
                                id INTEGER PRIMARY KEY,
                                petname VARCHAR(100) NOT NULL,
                                stage VARCHAR(100) NOT NULL,
                                step VARCHAR(100) NOT NULL,
                                type VARCHAR(100) NOT NULL,
                                subtype VARCHAR(100) NOT NULL, 
                                dm_x FLOAT NOT NULL,
                                dm_y FLOAT NOT NULL,
                                beam_type VARCHAR(100) NOT NULL,
                                timestamp TIMESTAMP NOT NULL,
                                experiment_id INTEGER NOT NULL,
                                FOREIGN KEY (experiment_id) REFERENCES experiment (id)
                                
                            );"""

# create / connect to db
database = "fibsem.db"
conn = create_connection(database)

# create tables
create_table(conn, SQL_CREATE_INTERACTIONS_TABLE)

In [ ]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe

EXPERIMENTS = [
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-02-09082023",
"/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-01-15082023",
]


df_exp = pd.read_sql("SELECT * FROM experiments", conn)
display(df_exp)


# 4, stage
# 5, detections
# 6, interactions

for PATH in EXPERIMENTS:

    dfs = calculate_statistics_dataframe(PATH)




    df = dfs[6]
    # # display(df)
    # # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    # print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # # replace NA with 0
    df = df.fillna(0)
    
    display(df[df.type=="MOVE"].head())

    # df.to_sql('interactions', conn, if_exists='append', index = False)

    print("------------------")

#### Upload all data to Database

In [ ]:
%load_ext autoreload
%autoreload 2
from autolamella import config as cfg
import os

import pandas as pd
from autolamella.structures import Experiment
from autolamella.tools.data import calculate_statistics_dataframe


import datetime
import sqlite3
from fibsem.db.util import *


# create / connect to db
database = "fibsem.db"
conn = create_connection(database)




# return

EXPERIMENTS = "/home/patrick/github/autolamella/autolamella/log/HANNAH-WAFFLE-O1-170823",

for PATH in EXPERIMENTS:

    print(PATH)   
    print("-------EXPERIMENT-------")

    exp = Experiment.load(os.path.join(PATH, "experiment.yaml"))
    df = exp.to_dataframe_v2()

    # format date as datetime
    df["date"] = pd.to_datetime(df["date"], unit="s")
    df["date"] = pd.to_datetime(df["date"], utc=True)

    # drop experiment_id
    df = df.drop(columns=["experiment_id", "num_lamella"])
    df["project_id"] = 1
    df["sample_id"] = 1
    df["user"] = "hannah"
    df["method"] = "waffle"
    
    display(df)

    # df.to_sql('experiments', conn, if_exists='append', index = False)

    print("------------------")

    print("-----------DATABASE EXPERIMENTS -------")
    df_exp = pd.read_sql("SELECT * FROM experiments", conn)
    display(df_exp)
    print("------------------")




    dfs = calculate_statistics_dataframe(PATH)

    print("-------HISTORY----------")
    df = dfs[1]

    # format date as datetime
    df["start"] = pd.to_datetime(df["start"], unit="s")
    df["start"] = pd.to_datetime(df["start"], utc=True)

    df["end"] = pd.to_datetime(df["start"], unit="s")
    df["end"] = pd.to_datetime(df["start"], utc=True)


    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])
    
    display(df.head())


    df.to_sql('history', conn, if_exists='append', index = False)

    print("------------------")


    print("-------STEPS----------")

    df = dfs[3]
    # display(df)
    # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())

    df.to_sql('steps', conn, if_exists='append', index = False)

    print("------------------")


    print("-------DETECTIONS----------")
    df = dfs[5]

    # # display(df)
    # # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())

    df.to_sql('detections', conn, if_exists='append', index = False)

    print("------------------")


    #### INTERACTIONS
    print("-------INTERACTIONS----------")
    df = dfs[6]

    # # # format date as datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    EXP_ID = df_exp[df_exp["name"]==df["exp_name"].iloc[0]]["id"].iloc[0]

    # print(EXP_ID)
    df["experiment_id"] = EXP_ID
    # # drop experiment_id
    df = df.drop(columns=["exp_id", "exp_name"])

    # # rename lamella to petname
    df = df.rename(columns={"lamella": "petname"})

    # # replace NA with 0
    df = df.fillna(0)
    
    display(df.head())

    df.to_sql('interactions', conn, if_exists='append', index = False)

    print("------------------")